In [ ]:
import pandas as pd
from datasets import load_dataset
from processor import Preprocessor
from similarity import SentenceTransformerSimilarity, BertSimilarity, TFIDFSimilarity
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
data_path = "cnamuangtoun/resume-job-description-fit"

In [3]:
datasets = load_dataset(data_path)

train_dataset = datasets['train']
test_dataset = datasets['test']

In [4]:
preprocessor = Preprocessor()
sentence_transformer = SentenceTransformerSimilarity("mixedbread-ai/mxbai-embed-large-v1")
bert_similarity = BertSimilarity()
tfidf_similarity = TFIDFSimilarity()

In [5]:
def compute_similarity(example):
    resume = preprocessor.preprocess(example["resume_text"])
    job_description = preprocessor.preprocess(example["job_description_text"])
    label = 0 if example["label"] == "No Fit" else 1

    tfidf = tfidf_similarity.similarity(resume, job_description)
    bert = bert_similarity.similarity(resume, job_description)
    transformer = sentence_transformer.similarity(resume, job_description)

    return {
        "resume": resume, "job_description": job_description, "label": label,
        "tfidf_similarity": tfidf, "bert_similarity": bert, "sentence_transformer_similarity": transformer
    }

In [6]:
test_dataset = test_dataset.map(compute_similarity)

Map: 100%|██████████| 1759/1759 [1:06:38<00:00,  2.27s/ examples]


In [14]:
test_dataset

Dataset({
    features: ['resume_text', 'job_description_text', 'label', 'resume', 'job_description', 'tfidf_similarity', 'bert_similarity', 'sentence_transformer_similarity'],
    num_rows: 1759
})

In [73]:
def convert_to_label(example):
    tfidf = 0 if example["tfidf_similarity"] < 0.5 else 1
    bert = 0 if example["bert_similarity"] < 0.5 else 1
    transformer = 0 if example["sentence_transformer_similarity"] < 0.65 else 1

    return {"tfidf": tfidf, "bert": bert, "transformer": transformer}


In [74]:
test_dataset = test_dataset.map(convert_to_label)

Map: 100%|██████████| 1759/1759 [00:00<00:00, 12958.71 examples/s]


In [75]:
test_dataset

Dataset({
    features: ['resume_text', 'job_description_text', 'label', 'resume', 'job_description', 'tfidf_similarity', 'bert_similarity', 'sentence_transformer_similarity', 'tfidf', 'bert', 'transformer'],
    num_rows: 1759
})

In [76]:
actual = test_dataset["label"]
bert_predict = test_dataset["bert"]
tfidf_predict = test_dataset["tfidf"]
transformer_predict = test_dataset["transformer"]

In [79]:
def evaluate(actual, predict):
    accuracy = accuracy_score(actual, predict)
    precision = precision_score(actual, predict)
    recall = recall_score(actual, predict)
    f1 = f1_score(actual, predict)
    
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [80]:
bert_evaluate = evaluate(actual, bert_predict)
tfidf_evaluate = evaluate(actual, tfidf_predict)
transformer_evaluate = evaluate(actual, transformer_predict)

In [81]:
print("bert evaluate : ", bert_evaluate)
print("tfidf evaluate : ", tfidf_evaluate)
print("transformer evaluate : ", transformer_evaluate)

bert evaluate :  {'accuracy': 0.5127913587265491, 'precision': 0.5127913587265491, 'recall': 1.0, 'f1': 0.6779406238256295}
tfidf evaluate :  {'accuracy': 0.4860716316088687, 'precision': 0.0, 'recall': 0.0, 'f1': 0.0}
transformer evaluate :  {'accuracy': 0.6031836270608301, 'precision': 0.5791925465838509, 'recall': 0.8270509977827051, 'f1': 0.6812785388127854}


In [82]:
data = {
    'BERT': bert_evaluate,
    'TF-IDF': tfidf_evaluate,
    'Transformer': transformer_evaluate
}

dataframe = pd.DataFrame(data)

In [40]:
dataframe

,BERT,TF-IDF,Transformer
accuracy,0.512791,0.486072,0.603184
precision,0.512791,0.000000,0.579193
recall,1.000000,0.000000,0.827051
f1,0.677941,0.000000,0.681279


In [41]:
dataframe.to_csv("evaluation.csv")